# SAS Job Execution - job execution functions
Job definitions are jobs that you can execute. You first create a job definition before executing it.  
The funtions in this use case execute jobs based on existing job definitions.

## Authentication

In [ ]:
#############################################
########## authentication ###################
#############################################

### with source("./R/authentication/auth_package.R") you will be able
### to load all functions here defined to facilitate other projects
### how to use example at the end of this file

## Refer to the authentication project

#Global variables to assign:
source("../authentication/get-access-token-r.r") 
sasserver  <- "http://your-server"
client_name <- "r_client" ## your client
client_secret <- "r_secret" ## your password
username <- "viya_user"
password <- "viya_password"

tokenDetailed <- authenticate(host = sasserver,
                              username = username,
                              password = password,
                              client_name = client_name,
                              client_secret = client_secret)

token <- tokenDetailed$access_token
token

## Import modules, variable assignment

In [ ]:
library("httr")
library("jsonlite")
updated_def1 <- readRDS("updated.Rda")$id # reading the file from job-definition-crud-r to get job id

# Variables to assign (uncomment and assign if you did not do so in the authenticaiton step above)
# sasserver  <- "http://your-server"
# client_name <- "r_client" ## create your client
# client_secret <- "r_secret" ## create your password
# username <- "viya_user"
# password <- "viya_password"

## Create *get job list* function

In [ ]:
#############################################
######### get job request list ##############
#############################################


### To execute a job you will need the
### `updated_def1` variable created on `job_definition-crud-r.r`

library("httr")
library("jsonlite")

get_jobRequest_list <- function(host, 
                                access_token, 
                                start = 0,
                                limit = 10,
                                filter = NULL,
                                verbose = FALSE) {
  
  url <- parse_url(host)
  url$path <- "/jobExecution/jobs"
  url$query <- list(
    start = start,
    limit = limit,
    filter = filter
  )
  
  response <- GET(
    url = build_url(url),
    add_headers(
      "accept"="application/vnd.sas.api+json",
      "authorization" = paste("Bearer", access_token)
    ),
    
    if(verbose) verbose()
  )
  
  stop_for_status(response)
  execList <- fromJSON(content(response, as = "text"))
  return(execList)
}

## Run the *get job definitions* function

In [ ]:
### protip: always that is possible use a filter
### otherwise the endpoint is going to do a full request
### and it is going to take a while

joblist <- get_jobRequest_list(sasserver, 
                         token,
                         start = 1, 
                         limit = 20,
                         filter = "eq(createdBy, 'sasdemo')"
                         )


joblist

## Create a *run job* function

In [ ]:
##################################################
### Submit a Job Definition for Execution ########
##################################################

execute_job_definition <- function(host, 
                                access_token, 
                                jobDefinitionId,
                                
                                ### following parameters overrides definitions
                                arguments = NULL,
                                ## persistant job needs createdBy, name, desc

                                verbose = FALSE) {
  
  url <- parse_url(host)
  url$path <- "/jobExecution/jobs"

  body <- toJSON(list(
    jobDefinitionUri = paste0("/jobDefinitions/definitions/", jobDefinitionId)
  ),
  auto_unbox = TRUE
  )
  
  response <- POST(
    url = build_url(url),
    add_headers(
      "accept"="application/vnd.sas.api+json",
      "Content-Type" = "application/vnd.sas.job.execution.job.request+json",
      "authorization" = paste("Bearer", access_token)
    ),
    
    body = body, 
    if(verbose) verbose()
  )
  
  stop_for_status(response)
  exec <- fromJSON(content(response, as = "text"))
  exec$etag <- cache_info(response)$etag
  return(exec)
}

## Run the *run job* function

In [ ]:
### execution is async
exec <- execute_job_definition(sasserver, 
                       token,
                       updated_def1 ## job created in job_definition.R
                       )
exec

## use defined parameters
exec_param <- execute_job_definition(sasserver, 
                               token,
                               updated_def1, ## job created in job_execution.R
                              list(AGE = 14)
                               )

exec_param$jobRequest$arguments

## Create a *get job execution state* function

In [ ]:
check_job_state <- function(host, 
                      access_token, 
                      executionId,
                      verbose = FALSE) {
  
  url <- parse_url(host)
  url$path <- paste0("/jobExecution/jobs/", executionId,"/state")
  
  response <- GET(
    url = build_url(url),
    add_headers(
      "accept"="text/plain",
       "authorization" = paste("Bearer", access_token)
    ),
    
    if(verbose) verbose()
  )
  
  stop_for_status(response)
  execState <- content(response, as = "text")
  return(execState)
}

## Run the *get job execution state* function

In [ ]:
execState <- check_job_state(sasserver, 
                             token,
                             exec$id)
execState

## Create a *get job execution details* function

In [ ]:
get_job_state <- function(host, 
                            access_token, 
                            executionId,
                            verbose = FALSE) {
  
  url <- parse_url(host)
  url$path <- paste0("/jobExecution/jobs/", executionId)
  
  response <- GET(
    url = build_url(url),
    add_headers(
      "accept"="application/vnd.sas.job.execution.job+json",
      "authorization" = paste("Bearer", access_token)
    ),
    
    if(verbose) verbose()
  )
  
  stop_for_status(response)
  execState <- fromJSON(content(response, as = "text"))
  return(execState)
}

## Run the *get job execution details* function

In [ ]:
execStateFull <- get_job_state(sasserver, 
                             token,
                             exec$id)

execStateFull$results ## output files
execStateFull$endTimeStamp
execStateFull$links
execStateFull$elapsedTime ## in seconds?
execStateFull$id

## Create an *update job exectuion request* function

In [ ]:
update_execute_job_definition <- function(host, 
                                   access_token, 
                                   jobReqId, ## requestId (exec)
                                   jobDefinitionId, ## definitionId
                                   etag,
                                   name = NULL,
                                   description = NULL,
                                   arguments = NULL,
                                   verbose = FALSE) {
  
  url <- parse_url(host)
  url$path <- paste0("/jobExecution/jobRequests/", jobReqId)
  
  body <- toJSON(list(
    id = jobReqId,
    name = name,
    description = description,
    jobDefinitionUri = paste0("/jobDefinitions/definitions/", jobDefinitionId),
    arguments = arguments
  ),
  auto_unbox = TRUE
  )
  
  response <- PUT( ### updating changes from POST to PUT
    url = build_url(url),
    add_headers(
      "accept"="application/vnd.sas.api+json",
      "Content-Type" = "application/vnd.sas.job.execution.job.request+json",
      "authorization" = paste("Bearer", access_token),
      "If-Match" = etag
    ),
    
    body = body, 
    if(verbose) verbose()
  )
  
  stop_for_status(response)
  exec <- fromJSON(content(response, as = "text"))
  return(exec)
}

## Run the *update job exectuion request* function

In [ ]:
updated_jobExec <- update_execute_job_definition(sasserver,
                              token,
                              jobReqId = exec$id,
                              jobDefinitionId = exec$jobRequest$jobDefinition$id,
                              etag = exec$etag,
                              name = "sashelp.class distribution",
                              description = "ods output with ager 14 cutoff",
                              arguments = list(AGE = "14"))

updated_jobExec

## Create a *delete job definition* function

In [ ]:
#### Delete a Definition
delete_job_execution <- function(host, 
                                  access_token,
                                  executionId,
                                  verbose = FALSE) {
  
  url <- parse_url(host)
  url$path <- paste0("/jobExecution/jobs/", executionId)
  
  response <- DELETE(
    url = build_url(url),
    add_headers(
      "accept"="application/json",
      "authorization" = paste("Bearer", access_token)
    ),
    if(verbose) verbose()
  )
  
  stop_for_status(response)
  
  if(response$status_code == 204){
    print(paste0("The job execution ", executionId," was successfully deleted."))
  } else {
    print(paste0("The job execution ", executionId," was not deleted."))
  }
  return(response)
}


## Run the *delete job definition* function

In [ ]:
response <- delete_job_execution(sasserver,
                     token,
                     execStateFull$id)